## Transfer Learning CNN

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import applications
from keras import optimizers

Using TensorFlow backend.


In [3]:
#declare the training and testing images, labels
train_images=[]
test_images=[]
train_labels=[]
test_labels=[]

#number of classes
num_classes=10

In [4]:
training_dir="10-monkey-species/training"
validation_dir="10-monkey-species/validation"

In [5]:
#Reading training data
total_images = 0
total_count = 0
image_size = 512
label = -1
training = os.listdir(training_dir)

for file in training:
    file_name = os.listdir(os.path.join(training_dir, file))
    if len(file_name)>0:
        label += 1
        print(file + " monkeys = " + str(len(file_name)))
        total_images += len(file_name)
        for image in file_name:
            path = os.path.join(training_dir, file, image)
            img = cv2.imread(path)
            img = cv2.resize(img, (image_size, image_size))
            img = np.array(img)
            train_images.append(img)
            train_labels.append(label)
            total_count += 1

len(train_labels)

n0 monkeys = 105
n1 monkeys = 111
n2 monkeys = 110
n3 monkeys = 122
n4 monkeys = 105
n5 monkeys = 113
n6 monkeys = 106
n7 monkeys = 114
n8 monkeys = 106
n9 monkeys = 106


1098

In [6]:
#Reading test data
total_images = 0
total_count = 0
image_size = 512
label = -1
test = os.listdir(validation_dir)

for file in test:
    file_name = os.listdir(os.path.join(validation_dir, file))
    if len(file_name)>0:
        label += 1
        print(file + " monkeys = " + str(len(file_name)))
        total_images += len(file_name)
        for image in file_name:
            path = os.path.join(validation_dir, file, image)
            img = cv2.imread(path)
            img = cv2.resize(img, (image_size, image_size))
            img = np.array(img)
            test_images.append(img)
            test_labels.append(label)
            total_count += 1

len(test_labels)

n0 monkeys = 26
n1 monkeys = 28
n2 monkeys = 27
n3 monkeys = 30
n4 monkeys = 26
n5 monkeys = 28
n6 monkeys = 26
n7 monkeys = 28
n8 monkeys = 27
n9 monkeys = 26


272

In [7]:
random.seed(1)
idx = list(range(len(train_images)))
shuffled = random.shuffle(idx)
new_train = [train_images[i] for i in idx]
new_train_labels = [train_labels[i] for i in idx]

In [8]:
#normalizing the data
new_train = np.array(new_train)
test_images = np.array(test_images)
new_train = new_train.astype('float32')/255
test_images = test_images.astype('float32')/255

new_train_labels = np.array(new_train_labels)
test_labels = np.array(test_labels)
new_train_labels = keras.utils.to_categorical(new_train_labels,num_classes)
test_labels = keras.utils.to_categorical(test_labels,num_classes)



In [9]:
#Building the CNN model
model=Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add((Conv2D(64, (3,3), activation='relu' )))
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 510, 510, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 64)        0         
__________

In [11]:
#Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
#Fit the model
model.fit(new_train, new_train_labels,validation_split=0.04, batch_size=32, epochs=7)

Instructions for updating:
Use tf.cast instead.
Train on 1054 samples, validate on 44 samples
Epoch 1/7
1054/1054 [==============================] - 239s 227ms/step - loss: 2.7678 - acc: 0.1243 - val_loss: 2.1756 - val_acc: 0.2500
Epoch 2/7
1054/1054 [==============================] - 225s 213ms/step - loss: 1.9912 - acc: 0.3008 - val_loss: 1.9435 - val_acc: 0.3409
Epoch 3/7
1054/1054 [==============================] - 224s 213ms/step - loss: 1.6318 - acc: 0.4488 - val_loss: 1.5477 - val_acc: 0.4318
Epoch 4/7
1054/1054 [==============================] - 225s 213ms/step - loss: 1.2592 - acc: 0.5854 - val_loss: 1.6592 - val_acc: 0.3864
Epoch 5/7
1054/1054 [==============================] - 224s 213ms/step - loss: 0.9511 - acc: 0.7011 - val_loss: 1.4893 - val_acc: 0.4773
Epoch 6/7
1054/1054 [==============================] - 225s 213ms/step - loss: 0.6915 - acc: 0.7932 - val_loss: 1.5107 - val_acc: 0.5227
Epoch 7/7
1054/1054 [==============================] - 224s 213ms/step - loss: 0.523

In [13]:
model.evaluate(test_images, test_labels)

272/272 [==============================] - 19s 70ms/step


[1.561158909517176, 0.5404411764705882]